In [40]:
#dependencies
import pandas as pd
import math


In [41]:
#Bring in the dataset
gss = pd.io.stata.read_stata('GSS1996.dta', convert_categoricals=False)
gss.to_csv('gss_converted.csv')

In [42]:
#drop all instances where angry event was not recalled
gss=gss.dropna(subset=["chnang7"])

In [43]:
#Recode DV to dummy
gss["pray_anger"]=gss["chnang7"]
gss.loc[gss.chnang7 == 2, 'pray_anger'] = 0

In [44]:
#Recode sex to dummy, reference category = "male"
gss["sex_r"]=gss["sex"]
gss.loc[gss.sex == 1, 'sex_r'] = 0
gss.loc[gss.sex == 2, 'sex_r'] = 1

In [48]:
#Recode race to dummy, reference category = "white"
gss["race_r"]=gss["race"]
gss.loc[gss.race == 1, 'race_r'] = 0
gss.loc[gss.race > 1, 'race_r'] = 1

In [55]:
#Recode degree to dummy, reference category = "has degree" (note that in original study, ref cat was NO degree)
gss["degree_r"]=gss["degree"]
gss.loc[gss.degree <=3, 'degree_r'] = 0
gss.loc[gss.degree > 3, 'degree_r'] = 1

In [66]:
#Recode region to dummy, reference category = "From South"
gss["region_r"]=gss["region"]
gss.loc[(gss.region <=4) | (gss.region >7), 'region_r'] = 0
gss.loc[(gss.region >=5) & (gss.region <=7), 'region_r'] = 1

array([0, 1], dtype=int8)